# practice

### torch

In [1]:
import sys
sys.path.insert(0, '/media/allen/mass/deep-learning-works/')
import os.path as osp
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import cv2
from PIL import Image 

def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)


In [160]:
a = torch.zeros(5)

In [161]:
a[:2] = 1

In [162]:
a

tensor([1., 1., 0., 0., 0.])

In [166]:
b = torch.rand(2,5,3,3)

In [167]:
b

tensor([[[[0.6503, 0.6152, 0.4890],
          [0.1189, 0.2309, 0.0899],
          [0.3494, 0.7865, 0.1823]],

         [[0.4616, 0.1415, 0.5507],
          [0.9596, 0.1763, 0.4888],
          [0.1882, 0.6741, 0.8927]],

         [[0.9268, 0.1363, 0.5525],
          [0.4181, 0.0297, 0.0133],
          [0.4920, 0.5906, 0.6938]],

         [[0.1990, 0.3720, 0.4805],
          [0.6621, 0.3741, 0.0948],
          [0.0486, 0.3189, 0.4798]],

         [[0.4911, 0.7329, 0.4378],
          [0.2039, 0.1529, 0.1911],
          [0.5809, 0.4505, 0.8809]]],


        [[[0.1419, 0.2710, 0.2101],
          [0.4964, 0.1999, 0.6706],
          [0.7238, 0.7500, 0.2556]],

         [[0.1978, 0.3311, 0.0759],
          [0.5856, 0.0893, 0.7760],
          [0.9430, 0.9755, 0.5500]],

         [[0.5911, 0.3892, 0.0068],
          [0.2641, 0.5770, 0.6128],
          [0.3581, 0.0309, 0.0222]],

         [[0.4907, 0.1947, 0.5099],
          [0.4262, 0.5634, 0.9962],
          [0.5116, 0.3693, 0.0088]],

        

In [168]:
a.view(1,5,1,1).expand_as(b) * b

tensor([[[[0.6503, 0.6152, 0.4890],
          [0.1189, 0.2309, 0.0899],
          [0.3494, 0.7865, 0.1823]],

         [[0.4616, 0.1415, 0.5507],
          [0.9596, 0.1763, 0.4888],
          [0.1882, 0.6741, 0.8927]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]],


        [[[0.1419, 0.2710, 0.2101],
          [0.4964, 0.1999, 0.6706],
          [0.7238, 0.7500, 0.2556]],

         [[0.1978, 0.3311, 0.0759],
          [0.5856, 0.0893, 0.7760],
          [0.9430, 0.9755, 0.5500]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

        

In [20]:
bn = nn.BatchNorm2d(3, affine=False)

In [21]:
a = torch.rand(1,3,4,4)

In [48]:
a.view(1,3,-1).mean(dim=2)

tensor([[0.4481, 0.4975, 0.5667]])

In [85]:
print(bn.running_mean)
print(bn.running_var)
_ = bn(a)
print(bn.running_mean)
print(bn.running_var)

tensor([0.4470, 0.4963, 0.5653])
tensor([0.0669, 0.1172, 0.0811])
tensor([0.4471, 0.4964, 0.5654])
tensor([0.0667, 0.1170, 0.0809])


In [119]:
class Conv(nn.Module):
    def __init__(self, inc, ouc):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(inc, ouc, kernel_size=3, stride=1, padding=1)
    def forward(self, x, *args, **kwargs):
        return self.conv(x)

class ChannelSelector(nn.Module):
    """
    Random channel # selection
    """
    def __init__(self, channel_number):
        super(ChannelSelector, self).__init__()
        self.channel_number = channel_number

    def forward(self, x, block_channel_mask=None, **kwargs):
        assert block_channel_mask is not None
        assert block_channel_mask.dim() == 2
        assert block_channel_mask.size(1) == x.size(1)
        n, c = block_channel_mask.shape
        block_channel_mask = block_channel_mask.view(n, c, 1, 1)
        return x * block_channel_mask.expand_as(x)

class test(nn.Module):
    def __init__(self, inc, ouc):
        super(test, self).__init__()   
        self.feat = nn.ModuleList([
            Conv(inc, ouc),
            ChannelSelector(ouc)
        ])
    def forward(self, x, block_channel_mask):
        for m in self.feat:
            if isinstance(m, ChannelSelector):
                x = m(x, block_channel_mask)
            else:
                x = m(x)        
        return x
    
model = test(3, 6)

In [122]:
import random

In [174]:
upper_flops = 100
bottom_flops = 50
flops_cuts = 7
flops_interval = (upper_flops - bottom_flops) / flops_cuts
  
flops_ranges = [max(upper_flops - i * flops_interval, 0) for i in range(flops_cuts)] + \
                    [max(upper_flops - i * flops_interval, 0) for i in range(flops_cuts)][::-1]

In [176]:
children_size = 10
children_pick_interval = 3
children_pick_ids = list(range(0, children_size, children_pick_interval)) + \
                         list(reversed(range(0, children_size, children_pick_interval)))

In [178]:
[6 if idx == 0 or idx == 3 else idx for idx in children_pick_ids]

[6, 6, 6, 9, 9, 6, 6, 6]

In [175]:
flops_ranges

[100.0,
 92.85714285714286,
 85.71428571428571,
 78.57142857142857,
 71.42857142857143,
 64.28571428571428,
 57.14285714285714,
 57.14285714285714,
 64.28571428571428,
 71.42857142857143,
 78.57142857142857,
 85.71428571428571,
 92.85714285714286,
 100.0]

In [181]:
ddd = [Conv(3, 6), ChannelSelector(6)]
model = nn.ModuleList(ddd)

In [180]:
model.extend([Conv(3, 6), ChannelSelector(6)])

ModuleList(
  (0): Conv(
    (conv): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (1): ChannelSelector()
  (2): Conv(
    (conv): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (3): ChannelSelector()
)

In [113]:
model[0]

Conv(
  (conv): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [121]:
x = torch.rand(1,3,4,4)
model(x, a)

tensor([[[[ 0.0467,  0.1172,  0.1571,  0.1464],
          [-0.1358,  0.1284,  0.0068,  0.5578],
          [-0.0778,  0.1979,  0.0706,  0.3085],
          [-0.1943, -0.3655,  0.0188, -0.0087]],

         [[-0.1031, -0.1963, -0.3407, -0.3737],
          [ 0.0419, -0.2389,  0.0441, -0.2988],
          [ 0.0370, -0.0433, -0.3050, -0.2678],
          [-0.3274, -0.3901, -0.2171, -0.3513]],

         [[ 0.0000, -0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000, -0.0000, -0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.0000],
          [ 0.0000, -0.0000, -0.0000, -0.0000]],

         [[-0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000],
          [-0.0000, -0.0000, -0.0000, -0.0000]],

         [[-0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  0.0000],
          [-0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000, -0.0000, -